# [The Kaggle G-Research Crypto Forecasting](https://www.kaggle.com/c/g-research-crypto-forecasting)

The idea is to predict crypto prices with the given. See link above for full details of the competition

# Import Dependencies

# Define Custom Functions

# Load the data

# EDA

# Minimal Viable Model

# Improvements

# Conclusions